In [ ]:
#Imports
import keras
import os
import numpy as np

In [ ]:
#Declare possible labels
data = []
labels = [
        "red",
        "green",
        "blue",
        "orange",
        "yellow",
        "pink",
        "purple",
        "brown",
        "grey",
        "black"
    ]

In [ ]:
#Load data and labels from clean_data.txt
filepath = "clean_data.txt"
if not os.path.isfile(filepath):
    print("File path {} does not exist. Exiting...".format(filepath))
    sys.exit()
with open(filepath) as fp:
    entry = []
    for line in fp:
        word = line.split(",")
        r = word[0]
        g = word[1]
        b = word[2]
        label = word[3].rstrip("\n")
        entry = [r, g, b, label]
        data.append(entry)

In [ ]:
#Prepare data
np.random.shuffle(data)
data = np.array(data)
xs = data[:,0:3]
xs = np.divide(xs.astype(np.float), 255) #normalize
print(np.shape(xs))

ys = data[:, 3]
ys = np.vectorize(labels.index)(ys)
print(np.shape(ys))


In [ ]:
#Seperate into training and testing data
XTrain = xs[0:5500,:]
YTrain = ys[0:5500]
YTrain = keras.backend.one_hot(YTrain, 10)

XTest = xs[5500:, :]
YTest = ys[5500:]
YTest = keras.backend.one_hot(YTest, 10)

In [ ]:
#Create architecture for model
model = keras.Sequential()
    
model.add(keras.layers.Dense(units=16, activation='sigmoid', input_shape=[3]))

model.add(keras.layers.Dense(units=10, activation='softmax'))

opt = keras.optimizers.SGD(learning_rate=0.2)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])


In [ ]:
#Train model
history = model.fit(x=XTrain,y=YTrain, epochs=500, validation_split=0.1)

In [ ]:
#Specify here what rgb values you want to test
r = 31
g = 38
b = 163

In [ ]:
#Convert to regularized numpy array for the model
predict = [[(r / 255), (g / 255), (b / 255)]]
predict = np.array(predict)

In [ ]:
#Make model prediction
prediction = model.predict(predict)
prediction = keras.backend.argmax(prediction, axis=-1)
prediction = labels[int(prediction)]
print(prediction)

In [ ]:
#Save model as h5 file
model.save( 'models/model.h5' )

In [ ]:
#Save model as tflite file
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.post_training_quantize = True
tflite_buffer = converter.convert()
open( 'android/model.tflite' , 'wb' ).write( tflite_buffer )

In [ ]:
#Plot loss and accuracy history of training
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()



In [ ]:
def displayResults(results):   
    loss = results[0]
    accuracy = results[1]
    precision = results[2]
    recall = results[3]
    f1_score = calculateF1(precision, recall)
    print('\n\nResults from evaluating on training data: \n')
    print('Loss', round(loss, 2))
    print('Accuracy', round(accuracy, 2))
    print('Precision: ', round(precision, 2))
    print('Recall: ', round(recall, 2))
    print('F1 Score: ', round(f1_score, 2))

In [ ]:
trainEval = model.evaluate(x=XTrain, y=YTrain)
displayResults(trainEval)

In [ ]:
testEval = model.evaluate(x=XTest, y=YTest)
displayResults(testEval)



In [ ]:
def calculateF1(precision, recall):
    return 2 * ((precision * recall)/ (precision + recall))

In [ ]:
model.evaluate(x=XTest, y=YTest)